In [1]:
import pandas as pd

caminho_arquivo = r"C:\Users\Caio do Carmo Carlos\Documents\Git\analise-lotofacil\dados\Lotofacil.xlsx"

# Carregar as primeiras planilhas e visualizar as primeiras linhas
xls = pd.ExcelFile(caminho_arquivo)
sheet_names = xls.sheet_names  # Ver quais abas existem
df = xls.parse(sheet_names[0])
df.head()

,Concurso,Data Sorteio,Bola1,Bola2,Bola3,Bola4,Bola5,Bola6,Bola7,Bola8,...,Rateio 13 acertos,Ganhadores 12 acertos,Rateio 12 acertos,Ganhadores 11 acertos,Rateio 11 acertos,Acumulado 15 acertos,Arrecadacao Total,Estimativa Prêmio,Acumulado sorteio especial Lotofácil da Independência,Observação
0,1,29/09/2003,2,3,5,6,9,10,11,13,...,"R$10,00",48807,"R$4,00",257593,"R$2,00","R$0,00","R$0,00","R$0,00","R$0,00",Estimativa de prêmio (15 ACERTOS) próximo conc...
1,2,06/10/2003,1,4,5,6,7,9,11,12,...,"R$10,00",81252,"R$4,00",478188,"R$2,00","R$0,00","R$0,00","R$0,00","R$0,00",ESTIMATIVA DE PRÊMIO PARA O PRÓXIMO CONCURSO (...
2,3,13/10/2003,1,4,6,7,8,9,10,11,...,"R$10,00",96244,"R$4,00",608211,"R$2,00","R$0,00","R$0,00","R$0,00","R$0,00",Estimativa de prêmio (15 ACERTOS) próximo conc...
3,4,20/10/2003,1,2,4,5,8,10,12,13,...,"R$10,00",123912,"R$4,00",706657,"R$2,00","R$0,00","R$0,00","R$0,00","R$0,00",Estimativa de prêmio (15 ACERTOS) próximo conc...
4,5,27/10/2003,1,2,4,8,9,11,12,13,...,"R$10,00",195636,"R$4,00",860992,"R$2,00","R$0,00","R$0,00","R$0,00","R$0,00",Estimativa de prêmio para o próximo concurso (...


In [2]:
# Renomear colunas para facilitar a manipulação
df.columns = [col.strip().replace(' ', '_').replace('/', '_').replace('á', 'a').replace('ç', 'c') for col in df.columns]

# Converter a coluna de data
df['Data_Sorteio'] = pd.to_datetime(df['Data_Sorteio'], dayfirst=True, errors='coerce')

# Garantir que as bolas sejam inteiras
for i in range(1, 16):
    df[f'Bola{i}'] = pd.to_numeric(df[f'Bola{i}'], errors='coerce')


In [3]:
# Criar a coluna com lista de dezenas
df['Dezenas'] = df[[f'Bola{i}' for i in range(1, 16)]].values.tolist()

# Criar a coluna com string das dezenas separadas por hífen (ex: "01-03-05-...-25")
df['Dezenas_str'] = df['Dezenas'].apply(lambda x: '-'.join(f'{int(i):02d}' for i in sorted(x)))


In [4]:
# 6. Soma das Dezenas Sorteadas

df['Soma'] = df['Dezenas'].apply(sum)

In [5]:
# 7. Quantidade de Pares e Ímpares

df['Qtd_Pares'] = df['Dezenas'].apply(lambda x: sum(1 for n in x if n % 2 == 0))
df['Qtd_Impares'] = df['Dezenas'].apply(lambda x: sum(1 for n in x if n % 2 != 0))

In [6]:
# 8. Contagem por Linha e Coluna do Volante

# Funções para contar dezenas por linha
def contar_linhas(dezenas, linha):
    return sum(1 for d in dezenas if (linha - 1)*5 < d <= linha*5)

# Funções para contar dezenas por coluna
def contar_colunas(dezenas, coluna):
    return sum(1 for d in dezenas if (d - 1) % 5 == (coluna - 1))

# Criar colunas com as contagens
for i in range(1, 6):
    df[f'Qtd_Linha_{i}'] = df['Dezenas'].apply(lambda x: contar_linhas(x, i))
    df[f'Qtd_Coluna_{i}'] = df['Dezenas'].apply(lambda x: contar_colunas(x, i))


In [7]:
# 9. Número de Repetições em Relação ao Sorteio Anterior

df['Qtd_Repetidos'] = df['Dezenas'].shift(1).combine(df['Dezenas'], 
                            lambda a, b: len(set(a) & set(b)) if isinstance(a, list) else 0)

In [8]:
# 10. Análise de somas e padrões pares/ímpares

import matplotlib.pyplot as plt
import seaborn as sns

# Calcular a soma das dezenas sorteadas em cada concurso
df['Soma_Dezenas'] = df['Dezenas'].apply(sum)

# Calcular a quantidade de pares e ímpares
def contar_pares_impares(dezenas):
    pares = sum(1 for d in dezenas if d % 2 == 0)
    impares = 15 - pares
    return pares, impares

df[['Qtd_Pares', 'Qtd_Impares']] = df['Dezenas'].apply(
    lambda x: pd.Series(contar_pares_impares(x))
)

In [9]:
# 12. Análise de Antecessores e Sucessores

from collections import Counter

antecessores = Counter()
sucessores = Counter()

for i in range(1, len(df)):
    dezenas_atuais = set(df.loc[i, 'Dezenas'])
    dezenas_anteriores = set(df.loc[i-1, 'Dezenas'])

    antecessores.update(dezenas_anteriores & dezenas_atuais)
    sucessores.update(dezenas_atuais & set(df.loc[i+1, 'Dezenas']) if i+1 < len(df) else [])

print("Top 10 Antecessores:")
print(antecessores.most_common(10))

print("\nTop 10 Sucessores:")
print(sucessores.most_common(10))


Top 10 Antecessores:
[(20, 1342), (10, 1316), (25, 1315), (11, 1282), (14, 1261), (13, 1256), (24, 1250), (5, 1235), (2, 1226), (4, 1224)]

Top 10 Sucessores:
[(20, 1341), (10, 1316), (25, 1315), (11, 1281), (14, 1261), (13, 1255), (24, 1249), (5, 1234), (2, 1226), (4, 1224)]


In [10]:
# 13. Regra de Associação entre Dezenas

from itertools import combinations
from collections import Counter

contador_pares = Counter()

for dezenas in df['Dezenas']:
    for par in combinations(sorted(dezenas), 2):
        contador_pares[par] += 1

pares_mais_comuns = contador_pares.most_common(10)

print("Top 10 pares de dezenas que mais aparecem juntos:")
for par, freq in pares_mais_comuns:
    print(f"Pares {par[0]} e {par[1]} apareceram juntos {freq} vezes")


Top 10 pares de dezenas que mais aparecem juntos:
Pares 10 e 20 apareceram juntos 1302 vezes
Pares 11 e 20 apareceram juntos 1302 vezes
Pares 10 e 25 apareceram juntos 1298 vezes
Pares 13 e 20 apareceram juntos 1288 vezes
Pares 10 e 11 apareceram juntos 1278 vezes
Pares 20 e 25 apareceram juntos 1278 vezes
Pares 10 e 14 apareceram juntos 1275 vezes
Pares 14 e 20 apareceram juntos 1274 vezes
Pares 9 e 10 apareceram juntos 1267 vezes
Pares 24 e 25 apareceram juntos 1264 vezes


In [11]:
df.head()

,Concurso,Data_Sorteio,Bola1,Bola2,Bola3,Bola4,Bola5,Bola6,Bola7,Bola8,...,Qtd_Linha_2,Qtd_Coluna_2,Qtd_Linha_3,Qtd_Coluna_3,Qtd_Linha_4,Qtd_Coluna_4,Qtd_Linha_5,Qtd_Coluna_5,Qtd_Repetidos,Soma_Dezenas
0,1,2003-09-29,2,3,5,6,9,10,11,13,...,3,1,3,4,3,3,3,4,0,199
1,2,2003-10-06,1,4,5,6,7,9,11,12,...,3,2,4,2,3,4,2,3,9,185
2,3,2003-10-13,1,4,6,7,8,9,10,11,...,5,3,3,2,3,4,2,2,11,182
3,4,2003-10-20,1,2,4,5,8,10,12,13,...,2,3,2,4,4,3,3,3,9,197
4,5,2003-10-27,1,2,4,8,9,11,12,13,...,2,2,4,3,3,4,3,3,11,202


In [12]:
df.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['Concurso', 'Data_Sorteio', 'Bola1', 'Bola2', 'Bola3', 'Bola4', 'Bola5',
       'Bola6', 'Bola7', 'Bola8', 'Bola9', 'Bola10', 'Bola11', 'Bola12',
       'Bola13', 'Bola14', 'Bola15', 'Ganhadores_15_acertos', 'Cidade___UF',
       'Rateio_15_acertos', 'Ganhadores_14_acertos', 'Rateio_14_acertos',
       'Ganhadores_13_acertos', 'Rateio_13_acertos', 'Ganhadores_12_acertos',
       'Rateio_12_acertos', 'Ganhadores_11_acertos', 'Rateio_11_acertos',
       'Acumulado_15_acertos', 'Arrecadacao_Total', 'Estimativa_Prêmio',
       'Acumulado_sorteio_especial_Lotofacil_da_Independência', 'Observacão',
       'Dezenas', 'Dezenas_str', 'Soma', 'Qtd_Pares', 'Qtd_Impares',
       'Qtd_Linha_1', 'Qtd_Coluna_1', 'Qtd_Linha_2', 'Qtd_Coluna_2',
       'Qtd_Linha_3', 'Qtd_Coluna_3', 'Qtd_Linha_4', 'Qtd_Coluna_4',
       'Qtd_Linha_5', 'Qtd_Coluna_5', 'Qtd_Repetidos', 'Soma_Dezenas'],
      dtype='object')>

In [13]:
## Dezenas mais sorteadas no últimos N concursos

import pandas as pd
from collections import Counter

# Número de concursos recentes que você deseja considerar
n = 50  # pode ajustar conforme quiser

# Ordena o DataFrame pelo número do concurso em ordem decrescente e seleciona os últimos n concursos
df_ultimos_n = df.sort_values(by='Concurso', ascending=False).head(n)

# Lista das colunas de dezenas
col_dezenas = [f'Bola{i}' for i in range(1, 16)]

# Junta todas as dezenas dos últimos n concursos
todas_dezenas = df_ultimos_n[col_dezenas].values.flatten()

# Conta a frequência de cada dezena
frequencia_dezenas = Counter(todas_dezenas)

# Converte para DataFrame e ordena
frequencia_df = pd.DataFrame(frequencia_dezenas.items(), columns=['Dezena', 'Frequencia'])
frequencia_df = frequencia_df.sort_values(by='Frequencia', ascending=False).reset_index(drop=True)

# Exibe as 25 mais frequentes
print(frequencia_df.head(25))


    Dezena  Frequencia
0        1          36
1       14          34
2       11          34
3        8          34
4        2          33
5       24          33
6       22          32
7       15          31
8       18          31
9        7          31
10      23          30
11      17          30
12       3          30
13      10          30
14       9          29
15      13          29
16       6          29
17      25          29
18      16          28
19      19          28
20      20          27
21       5          26
22       4          26
23      21          25
24      12          25


In [14]:
from itertools import combinations
from collections import Counter

# Filtrar últimos n concursos
n = 50
df_n = df.tail(n)

# Extrair as colunas de bolas e calcular pares
col_bolas = [f'Bola{i}' for i in range(1, 16)]
pares_freq = Counter()

for _, row in df_n[col_bolas].iterrows():
    pares = combinations(sorted(row.values), 2)
    pares_freq.update(pares)

top_pares = pares_freq.most_common(20)
print("Top pares mais frequentes nos últimos", n, "concursos:")
for par, freq in top_pares:
    print(f"{par}: {freq}")


Top pares mais frequentes nos últimos 50 concursos:
(1, 2): 25
(1, 13): 25
(8, 24): 25
(1, 14): 24
(1, 3): 24
(2, 11): 24
(1, 10): 23
(2, 14): 23
(14, 25): 23
(1, 8): 23
(1, 9): 23
(8, 11): 23
(8, 15): 23
(11, 14): 23
(14, 24): 23
(1, 7): 22
(1, 18): 22
(1, 22): 22
(1, 23): 22
(7, 14): 22


In [15]:
from collections import defaultdict

antecessores = defaultdict(int)
sucessores = defaultdict(int)

for i in range(1, len(df_n)):
    atual = set(df_n.iloc[i][col_bolas])
    anterior = set(df_n.iloc[i - 1][col_bolas])
    proximo = set(df_n.iloc[i + 1][col_bolas]) if i + 1 < len(df_n) else set()

    for dezena in atual:
        antecessores[dezena] += len([d for d in anterior if d == dezena])
        sucessores[dezena] += len([d for d in proximo if d == dezena])

print("Top antecessores:", sorted(antecessores.items(), key=lambda x: x[1], reverse=True)[:20])
print("Top sucessores:", sorted(sucessores.items(), key=lambda x: x[1], reverse=True)[:20])


Top antecessores: [(1, 26), (11, 26), (8, 24), (14, 23), (7, 21), (24, 20), (2, 20), (15, 19), (18, 19), (22, 18), (17, 17), (3, 17), (9, 16), (13, 16), (25, 16), (6, 16), (16, 15), (19, 15), (10, 15), (23, 14)]
Top sucessores: [(11, 26), (1, 25), (8, 24), (14, 22), (7, 21), (24, 20), (2, 20), (15, 19), (18, 18), (17, 17), (22, 17), (3, 17), (9, 16), (25, 16), (6, 16), (13, 15), (10, 15), (16, 14), (19, 14), (23, 14)]


In [16]:
decadas = {'1-10': 0, '11-20': 0, '21-25': 0}
for _, row in df_n[col_bolas].iterrows():
    for dez in row.values:
        if dez <= 10:
            decadas['1-10'] += 1
        elif dez <= 20:
            decadas['11-20'] += 1
        else:
            decadas['21-25'] += 1

print("Ocorrência por década:", decadas)


Ocorrência por década: {'1-10': 304, '11-20': 297, '21-25': 149}


In [17]:
from collections import Counter

somas = df_n['Soma_Dezenas'].tolist()
contagem_somas = Counter(somas)
top_somas = contagem_somas.most_common(5)
print("Top somas mais frequentes:", top_somas)


Top somas mais frequentes: [(176, 3), (191, 3), (192, 3), (196, 2), (213, 2)]


In [18]:
# Dezena | Critérios cruzados | Frequência, coocorrência, antecessor, década
dezenas_fortes = [1, 2, 8, 11, 14, 15, 18, 7, 24, 13, 3, 10, 9, 25, 16, 22, 17, 23]
print(f"Base de 18 dezenas selecionadas: {sorted(dezenas_fortes)}")


Base de 18 dezenas selecionadas: [1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 22, 23, 24, 25]


In [22]:
import random

def contar_decadas(jogo):
    d1 = len([d for d in jogo if d <= 10])
    d2 = len([d for d in jogo if 11 <= d <= 20])
    d3 = len([d for d in jogo if d >= 21])
    return d1, d2, d3

# Base forte com 18 dezenas estratégicas
dezenas_fortes = [1, 2, 8, 11, 14, 15, 18, 7, 24, 13, 3, 10, 9, 25, 16, 22, 17, 23]
dezenas_prioritarias = [1, 2, 8, 11, 14]  # precisam aparecer com força

jogos = []
tentativas = 0
max_tentativas = 20000  # maior tolerância

while len(jogos) < 30 and tentativas < max_tentativas:
    jogo = sorted(random.sample(dezenas_fortes, 15))
    soma = sum(jogo)
    d1, d2, d3 = contar_decadas(jogo)
    num_prio = len(set(dezenas_prioritarias).intersection(jogo))

    if (
        170 <= soma <= 200 and     # margem de soma maior
        6 <= d1 <= 10 and          # 1ª década mais flexível
        4 <= d2 <= 7 and           # 2ª também
        0 <= d3 <= 4 and           # 3ª até 4 dezenas
        num_prio >= 3             # pelo menos 3 das top 5 quentes
    ):
        if jogo not in jogos:
            jogos.append(jogo)
    tentativas += 1

# Exibir os jogos gerados
for i, jogo in enumerate(jogos, 1):
    print(f"Jogo {i:02d}: {jogo} | Soma: {sum(jogo)}")


Jogo 01: [1, 2, 3, 7, 8, 9, 11, 13, 14, 15, 16, 17, 22, 23, 25] | Soma: 186
Jogo 02: [1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 17, 18, 22, 25] | Soma: 175
Jogo 03: [1, 2, 3, 7, 8, 10, 11, 13, 14, 15, 16, 18, 22, 23, 24] | Soma: 187
Jogo 04: [1, 2, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 22, 24, 25] | Soma: 199
Jogo 05: [1, 2, 3, 7, 9, 10, 11, 13, 15, 17, 18, 22, 23, 24, 25] | Soma: 200
Jogo 06: [1, 2, 3, 8, 9, 10, 11, 13, 15, 16, 17, 22, 23, 24, 25] | Soma: 199
Jogo 07: [1, 2, 3, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 23, 25] | Soma: 182
Jogo 08: [1, 2, 3, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 24, 25] | Soma: 186
Jogo 09: [1, 2, 3, 7, 8, 9, 10, 13, 14, 15, 16, 18, 22, 23, 25] | Soma: 186
Jogo 10: [1, 2, 3, 7, 8, 9, 10, 11, 14, 15, 17, 18, 22, 24, 25] | Soma: 186
Jogo 11: [1, 2, 3, 7, 8, 10, 11, 13, 14, 15, 16, 17, 22, 23, 25] | Soma: 187
Jogo 12: [1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 17, 22, 24, 25] | Soma: 181
Jogo 13: [1, 2, 3, 7, 8, 9, 10, 11, 13, 15, 16, 18, 22, 23, 24] | Soma: 182
Jogo 1